In [ ]:
import pandas as pd
import functools as ft
import utm
import numpy as np
from os import listdir
import functools as ft
from os.path import isfile, join

import sys
sys.path.append("..")
from tools import data_path, output_path

### Data origin: 
 Files were downloaded by hand over the data provider. Not reproducible without original data.

In [9]:
mypath = data_path + 'bavaria/fluesse-abfluss'

In [10]:
onlyfiles = [mypath + "/" + f for f in listdir(mypath) if isfile(join(mypath, f))]

In [11]:
def parse_table_bavaria(p):
    meta = pd.read_csv(p,nrows=5, sep=";")
    coords = pd.read_csv(p,skiprows=6,nrows=0, sep=";").columns
    H = pd.read_csv(p,skiprows=7,nrows=0, sep=";").columns
    data = pd.read_csv(p,skiprows=8, sep=";")
    #data["Abfluss [m³/s]"] = data["Abfluss [m³/s]"].str.replace(",",".").astype(float)
    data["Datum"] = pd.to_datetime(data["Datum"])
    names = list(meta["Quelle:"].str[:-1].values)
    val = list(meta["Bayerisches Landesamt für Umwelt, www.gkd.bayern.de"].values)
    names.append(coords[0])
    names.append(coords[2])
    val.append(coords[1])
    val.append(coords[3])
    names.append("coord meta")
    val.append(coords[4])
    names.append(H[0])
    val.append(H[1])
    meta = pd.DataFrame(val,index = names, columns = [p.split("/")[-1].split("_")[0]])
    data.rename(columns={"Abfluss [m³/s]": p.split("/")[-1].split("_")[0]}, inplace=True)
    data.index = data["Datum"]
    data.drop(columns=["Prüfstatus", "Datum"], inplace=True)
    return data, meta

In [12]:
# some stations are have no data available.
data_stack = []
meta_stack = []
for x in onlyfiles:
    try:
        a,b = parse_table_bavaria(x)
        data_stack.append(a)
        meta_stack.append(b)
    except:
        print(x)

/home/datasets4/stein/rivers/raw_data/bavaria/fluesse-abfluss/24121005_01.01.2019_31.12.2023_ezw_0.csv
/home/datasets4/stein/rivers/raw_data/bavaria/fluesse-abfluss/24116005_01.01.2019_31.12.2023_ezw_0.csv
/home/datasets4/stein/rivers/raw_data/bavaria/fluesse-abfluss/.~lock.12183005_01.01.2019_31.12.2023_ezw_0.csv#


In [13]:
meta  = ft.reduce(lambda left, right: pd.merge(left, right,how="outer", left_index=True, right_index=True), meta_stack).T
data =  ft.reduce(lambda left, right: pd.merge(left, right,how="outer", left_index=True, right_index=True), data_stack)
data.columns = [x +"_ba" for x in data.columns]

In [14]:
for x in data.columns:
    try:
        data[x] = data[x].str.replace(",", ".")
    except:
        print(x)

24064003_ba
10078000_ba
18005000_ba
24022003_ba
18008008_ba
10062000_ba
10092000_ba
10056302_ba
10088003_ba
24050009_ba


In [16]:
output_path

NameError: name 'output_path' is not defined

In [20]:
data = data.astype(float)
data.to_csv(output_path + "bavaria_processed.csv")

## Bring to standard format

In [28]:
# coordinates
# same as thuringia parsing
for ind, line in meta.iterrows():
    try:
        meta.loc[ind, ["Ostwert:", "Nordwert:"]] = utm.to_latlon(
            int(line["Ostwert:"]), int(line["Nordwert:"]), 32, "N"
        )
    except:
        print(ind)
        pass

In [29]:
meta = meta[["Gewässer","Ostwert:", "Nordwert:", "Pegelnullpunktshöhe:"]]
meta.columns = ["R", "X", "Y", "H"]
meta["D"] = np.nan

In [30]:
meta["H"] = meta["H"].str.replace(" m NHN (DHHN2016)", "", regex=False).str.replace(
    " m NN (DHHN12)", "", regex=False
).str.replace(
    " m ü. NN", "", regex=False
).str.replace(
    " m NN (Alt_System)", "", regex=False
).str.replace(
    " m NHN (DHHN92)", "", regex=False
).str.replace(",", ".").astype(float)

In [31]:
meta.isnull().sum()

R      0
X      0
Y      0
H      1
D    503
dtype: int64

In [32]:
# Set everything to -1 (for not determined)
# Then set all values that are available to 0 (real data.)
meta["QD"] = -1
meta["QH"] = -1
meta["QX"] = -1
meta["QY"] = -1
meta["QR"] = -1

In [33]:
meta.loc[~(meta["X"].isnull()), "QX"] = 0
meta.loc[~(meta["Y"].isnull()), "QY"] = 0
meta.loc[~(meta["D"].isnull()), "QD"] = 0
meta.loc[~(meta["R"].isnull()), "QR"] = 0
meta.loc[~(meta["H"].isnull()), "QH"] = 0

In [ ]:
meta.to_csv(output_path + "processed/bavaria_meta_data.csv")

: 